In [1]:
import sys

# in order to import the modules located at the root directory
sys.path.append("..")

In [2]:
import logging
from importlib import reload  # Not needed in Python 2

reload(logging)
logging.basicConfig(
    format="%(message)s",
    level=logging.DEBUG,
)

LOGGER = logging.getLogger(__name__)

In [3]:
from mrs.utils import SessionBuilder
from mrs.finetune_dataset import FinetuneDataset

builder = SessionBuilder(style="formal")
train_dataset = FinetuneDataset(builder=builder, train=True)
valid_dataset = FinetuneDataset(builder=builder, train=False)

Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Number of sessions: 236
https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Number of sessions: 236


In [4]:
len(train_dataset), len(valid_dataset)

(3020, 214)

In [5]:
print(train_dataset[5]["session_tokens"][0])
print(train_dataset.tokenizer.decode(train_dataset[5]["session_tokens"][0]))

[0, 7003, 2138, 26, 12736, 2075, 2182, 35, 5688, 2259, 2180, 1378, 18941, 5971, 35, 32000, 1545, 2116, 7020, 7003, 2138, 30581, 1750, 2318, 4390, 2043, 1380, 2227, 2182, 18, 32000, 3676, 4358, 2116, 1039, 2073, 7003, 2116, 3842, 6961, 35, 32000, 7070, 1236, 12190, 18, 14008, 7003, 2170, 2182, 18, 32000, 4292, 3676, 4030, 7003, 2116, 3842, 6961, 35, 32000, 2, 1891, 1236, 12190, 18, 4842, 2170, 4516, 1122, 2886, 10283, 18]
[CLS] 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요? <SEP> [SEP] 한 살입니다. 작년에 분양 받았어요.


In [17]:
from torch.utils.data import DataLoader
from mrs.finetune_dataset import FinetuneDatasetCollator

train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=FinetuneDatasetCollator(
        pad_idx=train_dataset.tokenizer.pad_token_id, max_length=99999
    ),
)

In [18]:
batch = next(iter(train_loader))

In [19]:
batch["batch_input_tokens"].shape, batch["batch_input_attentions"].shape, batch[
    "batch_labels"
].shape

(torch.Size([10, 65]), torch.Size([10, 65]), torch.Size([10]))

In [25]:
print(batch["batch_labels"])
print(train_loader.dataset.tokenizer.decode(batch["batch_input_tokens"][0, :]))
print(train_loader.dataset.tokenizer.decode(batch["batch_input_tokens"][5, :]))

tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> [SEP] 고양이를 6마리나요? 키우는거 안 힘드세요? [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> [SEP] 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
